# Creating and inserting data

## Setup

In [1]:
import os
import ibis

ibis.options.interactive = True

connection = ibis.sqlite.connect(os.path.join('data', 'geography.db'))

/nix/store/w8dn93jmynfm6h6rag9zlz8339krczzj-python3-3.9.9-env/lib/python3.9/site-packages/pyproj/__init__.py:91: UserWarning: Valid PROJ data directory not found. Either set the path using the environmental variable PROJ_LIB or with `pyproj.datadir.set_data_dir`.
  warnings.warn(str(err))


## Creating new tables from Ibis expressions


Suppose you have an Ibis expression that produces a table:

In [2]:
countries = connection.table('countries')

continent_name = (countries.continent
                           .case()
                           .when('AF', 'Africa')
                           .when('AN', 'Antarctica')
                           .when('AS', 'Asia')
                           .when('EU', 'Europe')
                           .when('NA', 'North America')
                           .when('OC', 'Oceania')
                           .when('SA', 'South America')
                           .else_(countries.continent)
                           .end()
                           .name('continent_name'))

expr = countries[countries.continent, continent_name].distinct()
expr

  continent continent_name
0        EU         Europe
1        AS           Asia
2        NA  North America
3        AF         Africa
4        AN     Antarctica
5        SA  South America
6        OC        Oceania

To create a table in the database from the results of this expression, use the connection's `create_table` method:

In [3]:
connection.create_table('continents', expr)

In [4]:
continents = connection.table('continents')
continents

  continent continent_name
0        EU         Europe
1        AS           Asia
2        NA  North America
3        AF         Africa
4        AN     Antarctica
5        SA  South America
6        OC        Oceania

Tables can be similarly dropped with `drop_table`

In [5]:
connection.drop_table('continents')

## Inserting data into existing tables


Some backends support inserting data into existing tables from expressions. This can be done using `connection.insert('table_name', expr)`.